# 初始化平行集群并在每个cpu中导入可能用到的库

In [ ]:
from ipyparallel import Client,Reference
rc = Client()
rc.ids
dv = rc[:]
with dv.sync_imports():
    import os
    import json
    from math import log, sqrt
    import re
    import csv
    import sys 
    import difflib
    import numpy as np
    from copy import deepcopy
    from scipy import sparse
    import pandas as pd
    from itertools import islice   #导入迭代器
    from collections import Counter
    import networkx as nx
    import pickle as pkl
    sys.path.append("/data1/cufe/students/2017210761wsy/code")
    import LDA_for_label
    import network_bulit
    from gensim import corpora
    from scipy.sparse import bsr_matrix, dok_matrix
    from gensim.models import word2vec
    from network_bulit import combine_tuple, trans_title_to_num
    import warnings
    warnings.filterwarnings("ignore")

# 在串行CPU中导入可能用到的库

In [ ]:
import os
import json
from math import log, sqrt
import re
import csv
import sys 
import difflib
import numpy as np
from copy import deepcopy
from scipy import sparse
import pandas as pd
from itertools import islice   #导入迭代器
from collections import Counter
import networkx as nx
import pickle as pkl
sys.path.append("/data1/cufe/students/2017210761wsy/code")
import LDA_for_label
import network_bulit
from gensim import corpora
from scipy.sparse import bsr_matrix, dok_matrix
from gensim.models import word2vec
from network_bulit import combine_tuple, trans_title_to_num
import warnings
warnings.filterwarnings("ignore")

# 串行中用到的三个小函数

In [11]:
#读取pkl
def read_pkl(path_pkl):
    x = open(path_pkl, 'rb')
    journals_dict = pkl.load(x,encoding='iso-8859-1')
    x.close()
    return journals_dict
#合并并行结果
def add_dv(dv_matrix):
    if isinstance(dv_matrix[0],pd.core.frame.DataFrame):
        result = pd.DataFrame(index = dv_matrix[0].index.tolist(), columns = dv_matrix[0].columns.tolist()).fillna(0)   
        for i in range(len(dv_matrix)):
            result = result + dv_matrix[i]
    elif isinstance(dv_matrix[0],pd.core.frame.DataFrame):
        result = {}
        for i in range(len(dv_matrix)):
            result.update(dv_matrix[i])       
    return result
#开启并行函数num为线程数
def lan_para(num=4):
    from ipyparallel import Client,Reference
    rc = Client()
    rc.ids
    dv = rc[:num]
    return dv

# 拓扑相似度家族函数

In [ ]:
##拓扑相似度
def Topology_similarity(graph,subnet_name,nod_pair_list):
    similarity_matrix = Topology_similarity_of_distance(graph,subnet_name)/2 +  Topology_similarity_of_com(graph,subnet_name)/2              
    return similarity_matrix

##依距离的
def Topology_similarity_of_distance(graph,subnet_name):
    ##距离相似度
    print(subnet_name+'\'s topological distance similarity starts to be constructed')
    short_distance = nx.all_pairs_shortest_path(graph)                            #自身的节点距离为1
    df_empty = pd.DataFrame(index=graph.nodes(), columns=graph.nodes()).fillna(0)  
    for index, row in short_distance:
        for link_nod, route in row.items():
            df_empty[index][link_nod]  = 1/(2**(len(route)-1))
    df_empty_T = deepcopy(df_empty.T) 
    df_dis_sim = df_empty_T + df_empty                                            #没有连接的就为零
    print(subnet_name+'\'s topological distance similarity has been finished')
    df_dis_sim = df_dis_sim.apply(lambda x: x/x.sum(),axis=1)
    return  df_dis_sim

##依共同引用的
def Topology_similarity_of_com(graph,subnet_name,nod_pair_list):
    ####################################
    def t_com_similarity(nod_pair):   
        index1 = nod_pair[0]
        index2 = nod_pair[1]
        global paper_group_dataframe,df_com_sim
        #############################
        def single_list(arr, target):
            return arr.count(target)
        #############################
        ##共同引用
        sum_cite = single_list(list(paper_group_dataframe.loc[index1]),1)+single_list(list(paper_group_dataframe.loc[index2]),1)
        com_cite = single_list(list(paper_group_dataframe.loc[index1]+paper_group_dataframe.loc[index2]),2)
        if sum_cite ==0:
            copq = 0
        else:
            copq = com_cite/(sum_cite-com_cite)
        ##共同被引用
        sum_cited = single_list(list(paper_group_dataframe.loc[:,index1]),1)+single_list(list(paper_group_dataframe.loc[:,index2]),1)
        com_cited = single_list(list(paper_group_dataframe.loc[:,index1]+paper_group_dataframe.loc[:,index2]),2)
        if sum_cited == 0:
            capq = 0
        else:
            capq = com_cited/(sum_cited-com_cited)
        df_com_sim.loc[index1,index2] = copq+capq
        df_com_sim.loc[index2,index1] = copq+capq     
    ####################################
    ##共同引用与共同被引用相似度
    print(subnet_name+'\'s topological citation similarity starts to be constructed')
    paper_group_matrix = nx.adjacency_matrix(graph).todense()
    paper_group_dataframe = pd.DataFrame(paper_group_matrix, index=graph.nodes(), columns=graph.nodes())
    df_com_sim = pd.DataFrame(index=graph.nodes(), columns=graph.nodes()).fillna(0)
    ##开启并行
    dv = lan_para()
    dv['paper_group_dataframe'] = paper_group_dataframe
    dv['df_com_sim'] = df_com_sim
    dv.map_sync(t_com_similarity, nod_pair_list)
    ##填充
    df_com_sim =  add_dv(dv['df_com_sim'])
    df_com_sim = df_com_sim.apply(lambda x: x/x.sum(),axis=1)
    print(subnet_name+'\'s topological citation similarity has been finished')
    return df_com_sim

# 属性相似度家族函数

In [ ]:
#属性相似度
def Attribute_similarity(paper_group, graph, nod_pair_list, path_dictionary, lda_model_path, w2v_model_path, subnet_name,num_topics=4):  
    
    df_publisher_sim = publisher(paper_group, graph, nod_pair_list, subnet_name)
    df_lda_sim = abstract(paper_group, graph, nod_pair_list, path_dictionary,lda_model_path, subnet_name, num_topics =4)
    df_key_sim = keywords_and_plus(paper_group, graph, nod_pair_list, w2v_model_path, subnet_name)
    
    return (df_publisher_sim+df_lda_sim+df_key_sim)/3

#发行商
def publisher(paper_group, graph, nod_pair_list, subnet_name):
    ##################################
    def pub_similarity(nod_pair):
        index1 = nod_pair[0]
        index2 = nod_pair[1]
        global paper_group,df_publisher_sim        
        if paper_group[index1]['ego_attribute']['publisher'] == paper_group[index2]['ego_attribute']['publisher']:
            df_publisher_sim.loc[index1,index2] = 1
            df_publisher_sim.loc[index2,index1] = 1
    ##################################    
    print(subnet_name+'\'s attribute publisher similarity starts to be constructed')        
    df_publisher_sim = pd.DataFrame(index = graph.nodes(), columns = graph.nodes()).fillna(0)
    ##开启并行
    dv = lan_para()
    dv['paper_group'] = paper_group
    dv['df_publisher_sim'] = df_publisher_sim
    dv.map_sync(pub_similarity, nod_pair_list)
    ##填充+归一化
    df_publisher_sim =  add_dv(dv['df_publisher_sim'])
    df_publisher_sim = df_publisher_sim.apply(lambda x: x/x.sum(),axis=1)       
    print(subnet_name+'\'s attribute publisher similarity has been finished')
    return df_publisher_sim

##摘要    
def abstract(paper_group, graph, nod_pair_list, path_dictionary,lda_model_path, subnet_name, num_topics =4):
    ########################################
    def abs_similarity(nod_pair):
        index1 = nod_pair[0]
        index2 = nod_pair[1]
        global df_lda_score,df_lda_sim
        M = (df_lda_score.loc[index1]+df_lda_score.loc[index2])/2
        D1 = (df_lda_score.loc[index1]*((df_lda_score.loc[index1]/M).apply(lambda x:log(x)))).sum(axis=0)
        D2 = (df_lda_score.loc[index2]*((df_lda_score.loc[index2]/M).apply(lambda x:log(x)))).sum(axis=0) 
        topic = 1- sqrt(D1 + D2)
        df_lda_sim.loc[index1,index2] = topic
        df_lda_sim.loc[index2,index1] = topic         
    ########################################
    print(subnet_name+'\'s attribute abstract similarity starts to be constructed')                
    ##载入LDA主题模型和字典
    dictionary = corpora.Dictionary.load(path_dictionary)
    ldamodel = read_pkl(lda_model_path)
    ##生成LDA结果储存矩阵
    df_lda_score = pd.DataFrame(index = graph.nodes(), columns = ["lda"+str(i) for i in range(num_topics)]).fillna(0.00001) #防止除零无意义
    for paper, item in paper_group.items():
        bow_vector = dictionary.doc2bow(convert_doc_to_wordlist(item['ego_attribute']['abstract']))
        for index, score in sorted(ldamodel[bow_vector], key=lambda tup: -1*tup[1]):
            df_lda_score.loc[paper,"lda"+str(index)] = score+0.00001                                                        #防止除零无意义
        print(paper)
    #Jensen-shannon散度
    df_lda_sim = pd.DataFrame(index = graph.nodes(), columns = graph.nodes()).fillna(0) 
    ##开启并行
    dv = lan_para()
    dv['df_lda_score'] = df_lda_score
    dv['df_lda_sim'] = df_lda_sim
    dv.map_sync(abs_similarity, nod_pair_list)
    ##填充+归一化
    df_lda_sim =  add_dv(dv['df_lda_sim'])
    df_lda_sim = df_lda_sim.apply(lambda x: x/x.sum(),axis=1)  
    print(subnet_name+'\'s attribute abstract similarity has been finished')
    return df_lda_sim

#关键词
def keywords_and_plus(paper_group, graph,nod_pair_list,w2v_model_path, subnet_name):
    ##################################
    def key_label(nod_label):
        r1 = u'[a-zA-Z]'
        index1 = nod_label[0]
        candidate = nod_label[1]
        global df_label,label_sum, w2v_model 
        for word in candidate:
            word_loc = label_sum.index(word)
            if word_loc<20:
                word_loc = 20
            for index2 in label_sum[word_loc-20:word_loc+20] :
                if difflib.SequenceMatcher(None, index2, word).quick_ratio() > 0.98:
                    df_label.loc[index1,index2] = 1        
                elif 0.80<difflib.SequenceMatcher(None, index2, word).quick_ratio()<0.98:
                    try:
                        if len(re.sub(r1, '', index2)+re.sub(r1, '', word))== 0:
                            if w2v_model.n_similarity(''.join(index2.split()),''.join(word.split())) >0.95:
                                df_label.loc[index1,index2] = 1
                        else:
                            if w2v_model.n_similarity(''.join(convert_doc_to_wordlist(index2)),''.join(convert_doc_to_wordlist(word)) ) >0.93:
                                df_label.loc[index1,index2] = 1
                    except:
                        pass
        print(index1)
    ##################################
    def weight_word(keyword):
        global word_weight,df_label
        word_weight[keyword] = 1/(log(df_label.loc[:,keyword].sum(axis=0))+0.01)            
    ##################################
    def key_similarity(nod_pair):
        index1 = nod_pair[0]
        index2 = nod_pair[1]
        global df_key_sim,df_lda_sim,word_weight          
        paper_array = df_label.loc[index1,] + df_label.loc[index2,]
        com_label = paper_array[paper_array==2]._stat_axis.values.tolist()
        com_label_cou = len(com_label)
        sim_sum = 0
        if com_label_cou > 0:
            for one_label in com_label:
                sim_sum += word_weight[one_label]                    
        df_key_sim[paper1, paper2] = sim_sum
        df_key_sim[paper2, paper1] = sim_sum        
    ##################################
    print(subnet_name+'\'s attribute keywords similarity starts to be constructed')                
    #所有标签集合
    label_sum = []
    for key, item in paper_group.items():
        label_sum.extend(item['ego_attribute']['keywords']+item['ego_attribute']['keyword_plus'])
    label_sum = sorted(list(set(label_sum)))        #按首字母排序
    ##构建二部网络矩阵
    df_label = pd.DataFrame(index = graph.nodes(), columns = label_sum).fillna(0)         
    ##判断词汇相似性利用w2v技术和字符串匹配,也就是相似度和距离
    w2v_model = read_pkl(w2v_model_path)
    ##构建nod_label_list
    nod_label_list = []
    for index1, item in paper_group.items():
        candidate = list(set(item['ego_attribute']['keywords']+item['ego_attribute']['keyword_plus']))
        nod_label_list.append([index1,candidate])
    ##开启并行
    dv = lan_para()
    dv['df_label'] = df_label
    dv['label_sum'] = label_sum
    dv['w2v_model'] = w2v_model
    dv.map_sync(key_label, nod_label_list)
    df_label =  add_dv(dv['df_label'])
    #单词计算权重，开启并行
    dv = lan_para()
    dv['df_label'] = df_label
    dv['word_weight'] = {}
    dv.map_sync(weight_word, label_sum)
    word_weight = add_dv(dv['word_weight'])
    #计算标签相似度
    df_key_sim = pd.DataFrame(index = graph.nodes(), columns = graph.nodes()).fillna(0) 
    #开启并行
    dv = lan_para()
    dv['df_key_sim'] = df_key_sim
    dv['df_label'] = df_label
    dv['word_weight'] = word_weight
    dv.map_sync(key_similarity, nod_pair_list)
    df_key_sim = add_dv(dv['df_key_sim'])        
    df_key_sim = df_key_sim.apply(lambda x: x/x.sum(),axis=1)       #归一化 
    print(subnet_name+'\'s attribute keywords similarity has been finished')                                
    return df_key_sim


# 相似度主函数

In [ ]:
#计算论文节点相似度
def paper_similarity(paper_cluster,path_dictionary, lda_model_path, w2v_model_path, path_df_prob_pkl, num_topics):
    ##拆分名字
    subnet_name = paper_cluster[0]
    paper_group = paper_cluster[1] 
    ##构造有向图
    print(subnet_name+'\'s probability matrix'+'starts to be constructed')
    graph = nx.DiGraph()
    graph.add_nodes_from(list(paper_group.keys()))    #注意这里加入了所有点
    paper_group_edge = combine_tuple(paper_group)     #不是所有点都具有连边
    graph.add_edges_from(paper_group_edge)            #加入了部分节点的连边
    ##构造nod_pair_list准备并行
    nod_pair_list = []
    paper_name_list = list(paper_group.keys())
    paper_name_list_c = deepcopy(paper_name_list)
    for index1 in paper_name_list:
        for index2 in paper_name_list_c:
            nod_pair_list.append([index1,index2])
        paper_name_list_c.remove(index1)
    ##构造拓扑相似度
    print(subnet_name+'\'s topology similarity'+'starts to be constructed')
    df_topology = Topology_similarity(graph,subnet_name,nod_pair_list)          #拓扑相似度
    print(subnet_name+'\'s topology similarity'+'has been finished')
    ##构造属性相似度
    print(subnet_name+'\'s attribute similarity'+'starts to be constructed')
    df_attribute = Attribute_similarity(paper_group, graph, nod_pair_list, path_dictionary, lda_model_path, w2v_model_path, subnet_name,num_topics=4) #属性相似度
    print(subnet_name+'\'s attribute similarity'+'has been finished')
    df_pro = 0.382*df_topology + 0.618*df_attribute   #加权
    #归一化
    df_prob = df_pro.apply(lambda x: x/x.sum(),axis=1)
    path_df_prob_pkl_gen = path_df_prob_pkl + subnet_name + '.pkl'
    df_prob_file = open(path_df_prob_pkl_gen, 'wb')
    pkl.dump(df_prob, df_prob_file)
    df_prob_file.close()
    print(subnet_name+'\'s probability matrix'+'has been finished')

#  运行

In [ ]:
if __name__ == "__main__": 

    path_paper_dict_pkl = '/data1/cufe/students/2017210761wsy/pkl/paper_dict.pkl'
    paper_dict = read_pkl(path_paper_dict_pkl)
    
    ##制作路径
    w2v_model_path = '/data1/cufe/students/2017210761wsy/model/w2v_model.pkl'
    lda_model_path = '/data1/cufe/students/2017210761wsy/model/lda_true.pkl'
    path_dictionary = '/data1/cufe/students/2017210761wsy/model/dictionary.pkl'
    path_df_prob_pkl = '/data1/cufe/students/2017210761wsy/pkl/df_paper_prob_'
    
    ##切分子网络
    paper_cluster = ('network',paper_dict)
    #尝试运行paper_similarity
    num_topics = 4
    paper_similarity(paper_cluster,path_dictionary, lda_model_path, w2v_model_path, path_df_prob_pkl, num_topics)